In [1]:
 ## pull in the pickle files we generate in step 1

In [1]:
#parameter tagged cell for use with shell script
year = 2021
quarter = 'QTR2'

In [5]:
import os
import pickle

# path to the root directory where all the pickle files are stored
directory_path = f'/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Pickle_Files/{year}/{quarter}'
all_data = []

# walk through directory_path, including subdirectories
for dirpath, dirnames, filenames in os.walk(directory_path):
    print(f"Processing directory: {dirpath}")
    for filename in filenames:
        if filename.endswith('.pkl'):
            full_filepath = os.path.join(dirpath, filename)
            print(f"  Loading file: {filename}")
            with open(full_filepath, 'rb') as file:
                data = pickle.load(file)
                all_data.append(data)  

# all_data contains the combined data from all pickle files.


Processing directory: /Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Pickle_Files/2021/QTR2
  Loading file: 2024-04-17_batch5.pkl
  Loading file: 2024-04-17_batch68.pkl
  Loading file: 2024-04-17_batch40.pkl
  Loading file: 2024-04-17_batch54.pkl
  Loading file: 2024-04-17_batch83.pkl
  Loading file: 2024-04-17_batch97.pkl
  Loading file: 2024-04-17_batch96.pkl
  Loading file: 2024-04-17_batch82.pkl
  Loading file: 2024-04-17_batch55.pkl
  Loading file: 2024-04-17_batch41.pkl
  Loading file: 2024-04-17_batch69.pkl
  Loading file: 2024-04-17_batch4.pkl
  Loading file: 2024-04-17_batch6.pkl
  Loading file: 2024-04-17_batch57.pkl
  Loading file: 2024-04-17_batch43.pkl
  Loading file: 2024-04-17_batch94.pkl
  Loading file: 2024-04-17_batch80.pkl
  Loading file: 2024-04-17_batch81.pkl
  Loading file: 2024-04-17_batch95.pkl
  Loading file: 2024-04-17_batch42.pkl
  Loading file: 2024-04-17_batch56.pkl
  Loading file: 2024-04-17_batch7.pkl
  Loading file: 2024-04-17_batch52.pkl
  Loadin

In [7]:
import re

directory_path = f'/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Pickle_Files/{year}/{quarter}'

# regular expression pattern to match four consecutive digits for the year
year_pattern = re.compile(r'\b\d{4}\b')
# regular expression pattern to match "Q" followed by a digit for the quarter
quarter_pattern = re.compile(r'\bQTR[1-4]\b')


year_match = year_pattern.search(directory_path)
quarter_match = quarter_pattern.search(directory_path)

if year_match:
    year = year_match.group()
    print("Year extracted:", year)
else:
    print("No year found in the string.")

if quarter_match:
    quarter = quarter_match.group()
    print("Quarter extracted:", quarter)
else:
    print("No quarter found in the string.")


Year extracted: 2021
Quarter extracted: QTR2


In [8]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


 ## NON DERIVATIVE SECTION

In [9]:
def all_data_length(nested_list):
    total_count = 0
    for sublist in nested_list:
        for inner_list in sublist:
            total_count += len(inner_list)
    return total_count

# assuming your list of lists of lists is named 'list_of_lists'
total_count = all_data_length(all_data)
print("Total number of elements:", total_count)

Total number of elements: 72797


In [ ]:
#each unit in "all data" is a seperate batch. meaning, each unit down is a seperate batch
#each unit under that is one filing, which is a another list. with three dataframes and two strings

In [10]:
import pandas as pd

flattened_data = [df for sublist in all_data for df in sublist]

for index, df in enumerate(flattened_data):
    try:
        df['code'] = df['code'].astype(str).str[0]
        df['direct_or_indirect'] = df['direct_or_indirect'].str[0]
        df['amount'] = df['amount'].astype(str).str.split('(').str[0]

        # regex pattern to match numeric values and exclude any text within parentheses
        numeric_regex = r'\$?([0-9,\.]+)'

        if len(df) > 1:  # if more than one row
            df['total_owned_after_trans'] = df['total_owned_after_trans'].astype(str).str.extract(numeric_regex).squeeze().str.replace(',', '')
            df['Price'] = df['Price'].astype(str).str.extract(numeric_regex).squeeze().str.replace(',', '').str.replace('$', '')
            df['amount'] = df['amount'].astype(str).str.extract(numeric_regex).squeeze().str.replace(',', '')
            # remove content within parentheses if it exists
            df['Price'] = df['Price'].str.split('(').str[0]
        else:  # if only one row
            df['total_owned_after_trans'] = df['total_owned_after_trans'].astype(str).str.extract(numeric_regex)[0].replace(',', '')
            df['Price'] = df['Price'].astype(str).str.extract(numeric_regex)[0].replace(',', '').replace('$', '')
            df['amount'] = df['amount'].astype(str).str.extract(numeric_regex)[0].replace(',', '')
            # remove content within parentheses if it exists
            df['Price'] = df['Price'].str.split('(').str[0]
    except KeyError as e:
        print(f"KeyError encountered at DataFrame index {index}: {e}")


In [11]:
for df in flattened_data: 
    if df.shape[1] > 11:
        column_mapping = {
        df.columns[0]: 'security_type',
        df.columns[1]: 'transaction_date',
        df.columns[2]: 'execution_date',
        df.columns[3]: 'code',
        df.columns[4]: 'V',
        df.columns[5]: 'amount',
        df.columns[6]: 'a_or_d',
        df.columns[7]: 'price',
        df.columns[8]: 'amount_owned_following',
        df.columns[9]: 'd_or_i',
        df.columns[10]: 'nature_of_ownership'
        }
        df.rename(columns=column_mapping, inplace=True)
        df.columns.values[[11, 12, 13, 14]] = ['url', 'reporting_date', 'ticker','name']
        df.drop(columns=['execution_date', 'V', 'nature_of_ownership'], inplace=True)


In [12]:
flattened_data[0]

security_type transaction_date code  amount a_or_d  price  \
0   Common Stock       05/13/2021    M  9318.0      A  47.68   
1   Common Stock       05/13/2021    S   100.0      D   98.4   
2   Common Stock       05/13/2021    S   100.0      D  98.39   
3   Common Stock       05/13/2021    S   100.0      D  98.38   
4   Common Stock       05/13/2021    S   500.0      D  98.37   
5   Common Stock       05/13/2021    S   192.0      D  98.36   
6   Common Stock       05/13/2021    S   100.0      D  98.35   
7   Common Stock       05/13/2021    S   100.0      D  98.33   
8   Common Stock       05/13/2021    S   938.0      D  98.32   
9   Common Stock       05/13/2021    S    88.0      D  98.31   
10  Common Stock       05/13/2021    S   100.0      D   98.3   
11  Common Stock       05/13/2021    S   100.0      D  98.29   
12  Common Stock       05/13/2021    S   100.0      D  98.28   
13  Common Stock       05/13/2021    S   400.0      D  98.27   
14  Common Stock       05/13/2021    S   510.0      D  98.26   
15  Common Stock       05/13/2021    S   397.0      D  98.25   
16  Common Stock       05/13/2021    S   100.0      D  98.23   
17  Common Stock       05/13/2021    S   100.0      D  98.22   
18  Common Stock       05/13/2021    S   200.0      D   98.2   
19  Common Stock       05/13/2021    S   100.0      D  98.19   
20  Common Stock       05/13/2021    S   300.0      D  98.18   
21  Common Stock       05/13/2021    S   300.0      D  98.17   
22  Common Stock              NaN    n     NaN    NaN    NaN   
23  Common Stock              NaN    n     NaN    NaN    NaN   
24  Common Stock              NaN    n     NaN    NaN    NaN   
25  Common Stock              NaN    n     NaN    NaN    NaN   
26  Common Stock              NaN    n     NaN    NaN    NaN   
27  Common Stock              NaN    n     NaN    NaN    NaN   
28  Common Stock              NaN    n     NaN    NaN    NaN   
29  Common Stock              NaN    n     NaN    NaN    NaN   

   amount_owned_following d_or_i  \
0             207885.1153      D   
1             207785.1153      D   
2             207685.1153      D   
3             207585.1153      D   
4             207085.1153      D   
5             206893.1153      D   
6             206793.1153      D   
7             206693.1153      D   
8             205755.1153      D   
9             205667.1153      D   
10            205567.1153      D   
11            205467.1153      D   
12            205367.1153      D   
13            204967.1153      D   
14            204457.1153      D   
15            204060.1153      D   
16            203960.1153      D   
17            203860.1153      D   
18            203660.1153      D   
19            203560.1153      D   
20            203260.1153      D   
21            202960.1153      D   
22                 1000.0      I   
23               2103.133      I   
24                 1000.0      I   
25                53800.0      I   
26                12558.0      I   
27               290397.0      I   
28               395989.0      I   
29              2023836.0      I   

                                                  url reporting_date  ticker  \
0   https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   UMBF    
1   https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   UMBF    
2   https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   UMBF    
3   https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   UMBF    
4   https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   UMBF    
5   https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   UMBF    
6   https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   UMBF    
7   https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   UMBF    
8   https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   UMBF    
9   https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   UMBF    
10  https://sec.gov/A

In [ ]:
# only care about outright purchases

In [13]:
df_with_p_case_insensitive = [df for df in flattened_data if df['code'].str.lower().eq('p').any()]

In [14]:
len(df_with_p_case_insensitive)

6545

In [15]:
modified_dataframes = flattened_data

In [16]:
import pandas as pd

summary_list_1 = []  # list to store the summary of each df

for index, df in enumerate(modified_dataframes):  # using enumerate to get the index
    if not df.empty:
        print(f"Processing DataFrame at index: {index}")
        # clean up 'amount', 'price', and 'amount_owned_following' columns
        df['amount'] = pd.to_numeric(df['amount'].astype(str).str.replace(',', '').str.extract(r'([\d.]+)', expand=False), errors='coerce')
        df['price'] = pd.to_numeric(df['price'].astype(str).str.replace(',', '').str.extract(r'([\d.]+)', expand=False), errors='coerce')
        df['amount_owned_following'] = pd.to_numeric(df['amount_owned_following'].astype(str).str.replace(',', '').str.split('(').str[0], errors='coerce')

        # initialize net amounts
        df['total_amount_bought_or_sold'] = 0
        df['total_dollar_bought_or_sold'] = 0

        # calculate net buy/sell based on the codes
        for index, row in df.iterrows():
            code = row['code']
            amount = row['amount']
            price = row['price']
            if pd.notnull(amount) and pd.notnull(price):  # check if 'amount' and 'price' are not NaN
                dollar_amount = amount * price
                if code == 'P':
                    df.at[index, 'total_amount_bought_or_sold'] = amount
                    df.at[index, 'total_dollar_bought_or_sold'] = dollar_amount
                elif code == 'S':
                    df.at[index, 'total_amount_bought_or_sold'] = -amount
                    df.at[index, 'total_dollar_bought_or_sold'] = -dollar_amount

        # create a summary for the current DataFrame
        summary = {
            'transaction_date': df['transaction_date'].iloc[0] if 'transaction_date' in df else None,
            'total_amount_bought_or_sold': df['total_amount_bought_or_sold'].sum(),
            'total_dollar_bought_or_sold': df['total_dollar_bought_or_sold'].sum(),
            'purchase_price': df['price'].iloc[0] if 'price' in df else None,
            'amount_owned_following': df['amount_owned_following'].max() if 'amount_owned_following' in df else None,
            'url': df['url'].iloc[-1] if 'url' in df else None,
            'reporting_date': df['reporting_date'].iloc[0] if 'reporting_date' in df else None,
            'ticker': df['ticker'].iloc[0] if 'ticker' in df else None,
            'name': df['name'].iloc[0] if 'name' in df else None
            
        }
        summary_list_1.append(summary)

# convert the list of summaries to a DataFrame
final_summary_df_1 = pd.DataFrame(summary_list_1)

# display the final DataFrame
final_summary_df_1


Processing DataFrame at index: 0
Processing DataFrame at index: 1
Processing DataFrame at index: 2
Processing DataFrame at index: 3
Processing DataFrame at index: 4
Processing DataFrame at index: 5
Processing DataFrame at index: 6
Processing DataFrame at index: 7
Processing DataFrame at index: 8
Processing DataFrame at index: 9
Processing DataFrame at index: 10
Processing DataFrame at index: 11
Processing DataFrame at index: 12
Processing DataFrame at index: 13
Processing DataFrame at index: 14
Processing DataFrame at index: 15
Processing DataFrame at index: 16
Processing DataFrame at index: 17
Processing DataFrame at index: 18
Processing DataFrame at index: 19
Processing DataFrame at index: 20
Processing DataFrame at index: 21
Processing DataFrame at index: 22
Processing DataFrame at index: 23
Processing DataFrame at index: 24
Processing DataFrame at index: 25
Processing DataFrame at index: 26
Processing DataFrame at index: 27
Processing DataFrame at index: 28
Processing DataFrame at 

transaction_date  total_amount_bought_or_sold  \
0           05/13/2021                      -4925.0   
1           05/13/2021                      -5859.0   
2           05/14/2021                      -5045.0   
3           05/14/2021                      -4360.0   
4           05/14/2021                       -304.0   
...                ...                          ...   
21870       06/23/2021                      -1667.0   
21871       05/27/2021                      -1000.0   
21872       05/27/2021                    -300000.0   
21873       05/17/2021                      -6000.0   
21874       05/13/2021                      -1935.0   

       total_dollar_bought_or_sold  amount_owned_following  \
0                       -484052.41              2023836.00   
1                       -574667.56              2023836.00   
2                       -495973.56              2023836.00   
3                       -429786.80              2023836.00   
4                        -30038.12              2023836.00   
...                            ...                     ...   
21870                   -103537.37                33284.00   
21871                    -39080.00                60206.00   
21872                   -225000.00               300000.00   
21873                  -1007340.00                35710.06   
21874                    -79528.50                21242.00   

                                                     url reporting_date  \
0      https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   
1      https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   
2      https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   
3      https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   
4      https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   
...                                                  ...            ...   
21870  https://sec.gov/Archives/edgar/data/1724566/00...     2021-06-23   
21871  https://sec.gov/Archives/edgar/data/1724577/00...     2021-05-27   
21872  https://sec.gov/Archives/edgar/data/1724601/00...     2021-06-01   
21873  https://sec.gov/Archives/edgar/data/1724611/00...     2021-05-19   
21874  https://sec.gov/Archives/edgar/data/1724761/00...     2021-05-14   

       ticker                 name  purchase_price  
0       UMBF      KEMPER J MARINER           47.68  
1       UMBF      KEMPER J MARINER           98.16  
2       UMBF      KEMPER J MARINER           98.19  
3       UMBF      KEMPER J MARINER           98.45  
4       UMBF      KEMPER J MARINER           98.78  
...       ...                  ...             ...  
21870   QTRX        Duffy David C.           62.11  
21871   HBCP       GUIDRY DANIEL G           39.08  
21872   EMPR   Pritchard Thomas W.            0.25  
21873    CDW        KEBO COLLIN B.          167.89  
21874   SANM       BILLINGER BRENT           41.10  

[21875 rows x 9 columns]

Processing DataFrame at index: 1189
Processing DataFrame at index: 1190
Processing DataFrame at index: 1191
Processing DataFrame at index: 1192
Processing DataFrame at index: 1193
Processing DataFrame at index: 1194
Processing DataFrame at index: 1195
Processing DataFrame at index: 1196
Processing DataFrame at index: 1197
Processing DataFrame at index: 1198
Processing DataFrame at index: 1199
Processing DataFrame at index: 1200
Processing DataFrame at index: 1201
Processing DataFrame at index: 1202
Processing DataFrame at index: 1203
Processing DataFrame at index: 1204
Processing DataFrame at index: 1205
Processing DataFrame at index: 1206
Processing DataFrame at index: 1207
Processing DataFrame at index: 1208
Processing DataFrame at index: 1209
Processing DataFrame at index: 1210
Processing DataFrame at index: 1211
Processing DataFrame at index: 1212
Processing DataFrame at index: 1213
Processing DataFrame at index: 1214
Processing DataFrame at index: 1215
Processing DataFrame at inde

Processing DataFrame at index: 1454
Processing DataFrame at index: 1455
Processing DataFrame at index: 1456
Processing DataFrame at index: 1457
Processing DataFrame at index: 1458
Processing DataFrame at index: 1459
Processing DataFrame at index: 1460
Processing DataFrame at index: 1461
Processing DataFrame at index: 1462
Processing DataFrame at index: 1463
Processing DataFrame at index: 1464
Processing DataFrame at index: 1465
Processing DataFrame at index: 1466
Processing DataFrame at index: 1467
Processing DataFrame at index: 1468
Processing DataFrame at index: 1469
Processing DataFrame at index: 1470
Processing DataFrame at index: 1471
Processing DataFrame at index: 1472
Processing DataFrame at index: 1473
Processing DataFrame at index: 1474
Processing DataFrame at index: 1475
Processing DataFrame at index: 1476
Processing DataFrame at index: 1477
Processing DataFrame at index: 1478
Processing DataFrame at index: 1479
Processing DataFrame at index: 1480
Processing DataFrame at inde

Processing DataFrame at index: 1721
Processing DataFrame at index: 1722
Processing DataFrame at index: 1723
Processing DataFrame at index: 1724
Processing DataFrame at index: 1725
Processing DataFrame at index: 1726
Processing DataFrame at index: 1727
Processing DataFrame at index: 1728
Processing DataFrame at index: 1729
Processing DataFrame at index: 1730
Processing DataFrame at index: 1731
Processing DataFrame at index: 1732
Processing DataFrame at index: 1733
Processing DataFrame at index: 1734
Processing DataFrame at index: 1735
Processing DataFrame at index: 1736
Processing DataFrame at index: 1737
Processing DataFrame at index: 1738
Processing DataFrame at index: 1739
Processing DataFrame at index: 1740
Processing DataFrame at index: 1741
Processing DataFrame at index: 1742
Processing DataFrame at index: 1743
Processing DataFrame at index: 1744
Processing DataFrame at index: 1745
Processing DataFrame at index: 1746
Processing DataFrame at index: 1747
Processing DataFrame at inde

...

Processing DataFrame at index: 2261
Processing DataFrame at index: 2262
Processing DataFrame at index: 2263
Processing DataFrame at index: 2264
Processing DataFrame at index: 2265
Processing DataFrame at index: 2266
Processing DataFrame at index: 2267
Processing DataFrame at index: 2268
Processing DataFrame at index: 2269
Processing DataFrame at index: 2270
Processing DataFrame at index: 2271
Processing DataFrame at index: 2272
Processing DataFrame at index: 2273
Processing DataFrame at index: 2274
Processing DataFrame at index: 2275
Processing DataFrame at index: 2276
Processing DataFrame at index: 2277
Processing DataFrame at index: 2278
Processing DataFrame at index: 2279
Processing DataFrame at index: 2280
Processing DataFrame at index: 2281
Processing DataFrame at index: 2282
Processing DataFrame at index: 2283
Processing DataFrame at index: 2284
Processing DataFrame at index: 2285
Processing DataFrame at index: 2286
Processing DataFrame at index: 2287
Processing DataFrame at inde

Processing DataFrame at index: 2490
Processing DataFrame at index: 2491
Processing DataFrame at index: 2492
Processing DataFrame at index: 2493
Processing DataFrame at index: 2494
Processing DataFrame at index: 2495
Processing DataFrame at index: 2496
Processing DataFrame at index: 2497
Processing DataFrame at index: 2498
Processing DataFrame at index: 2499
Processing DataFrame at index: 2500
Processing DataFrame at index: 2501
Processing DataFrame at index: 2502
Processing DataFrame at index: 2503
Processing DataFrame at index: 2504
Processing DataFrame at index: 2505
Processing DataFrame at index: 2506
Processing DataFrame at index: 2507
Processing DataFrame at index: 2508
Processing DataFrame at index: 2509
Processing DataFrame at index: 2510
Processing DataFrame at index: 2511
Processing DataFrame at index: 2512
Processing DataFrame at index: 2513
Processing DataFrame at index: 2514
Processing DataFrame at index: 2515
Processing DataFrame at index: 2516
Processing DataFrame at inde

Processing DataFrame at index: 2730
Processing DataFrame at index: 2731
Processing DataFrame at index: 2732
Processing DataFrame at index: 2733
Processing DataFrame at index: 2734
Processing DataFrame at index: 2735
Processing DataFrame at index: 2736
Processing DataFrame at index: 2737
Processing DataFrame at index: 2738
Processing DataFrame at index: 2739
Processing DataFrame at index: 2740
Processing DataFrame at index: 2741
Processing DataFrame at index: 2742
Processing DataFrame at index: 2743
Processing DataFrame at index: 2744
Processing DataFrame at index: 2745
Processing DataFrame at index: 2746
Processing DataFrame at index: 2747
Processing DataFrame at index: 2748
Processing DataFrame at index: 2749
Processing DataFrame at index: 2750
Processing DataFrame at index: 2751
Processing DataFrame at index: 2752
Processing DataFrame at index: 2753
Processing DataFrame at index: 2754
Processing DataFrame at index: 2755
Processing DataFrame at index: 2756
Processing DataFrame at inde

Processing DataFrame at index: 2976
Processing DataFrame at index: 2977
Processing DataFrame at index: 2978
Processing DataFrame at index: 2979
Processing DataFrame at index: 2980
Processing DataFrame at index: 2981
Processing DataFrame at index: 2982
Processing DataFrame at index: 2983
Processing DataFrame at index: 2984
Processing DataFrame at index: 2985
Processing DataFrame at index: 2986
Processing DataFrame at index: 2987
Processing DataFrame at index: 2988
Processing DataFrame at index: 2989
Processing DataFrame at index: 2990
Processing DataFrame at index: 2991
Processing DataFrame at index: 2992
Processing DataFrame at index: 2993
Processing DataFrame at index: 2994
Processing DataFrame at index: 2995
Processing DataFrame at index: 2996
Processing DataFrame at index: 2997
Processing DataFrame at index: 2998
Processing DataFrame at index: 2999
Processing DataFrame at index: 3000
Processing DataFrame at index: 3001
Processing DataFrame at index: 3002
Processing DataFrame at inde

transaction_date  total_amount_bought_or_sold  \
0          05/13/2021                        -4925   
1          05/13/2021                        -5859   
2          05/14/2021                        -5045   
3          05/14/2021                        -4360   
4          05/14/2021                         -304   
...               ...                          ...   
3140       04/30/2021                       -60000   
3141       05/18/2021                       -66745   
3142       05/13/2021                        -1500   
3143       05/18/2021                        -8500   
3144       05/07/2021                      -250000   

      total_dollar_bought_or_sold  amount_owned_following  \
0                         -484051               2023836.0   
1                         -574662               2023836.0   
2                         -495968               2023836.0   
3                         -429782               2023836.0   
4                          -30038               2023836.0   
...                           ...                     ...   
3140                      -692172                410071.0   
3141                      -750753                365071.0   
3142                       -87114                 33358.0   
3143                      -479360                 23358.0   
3144                    -15925606                955041.0   

                                                    url reporting_date  \
0     https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   
1     https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   
2     https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   
3     https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   
4     https://sec.gov/Archives/edgar/data/101382/000...     2021-05-17   
...                                                 ...            ...   
3140  https://sec.gov/Archives/edgar/data/1195839/00...     2021-05-04   
3141  https://sec.gov/Archives/edgar/data/1195839/00...     2021-05-20   
3142  https://sec.gov/Archives/edgar/data/1195856/00...     2021-05-18   
3143  https://sec.gov/Archives/edgar/data/1195856/00...     2021-05-20   
3144  https://sec.gov/Archives/edgar/data/1195948/00...     2021-05-11   

      ticker                   name  
0      UMBF        KEMPER J MARINER  
1      UMBF        KEMPER J MARINER  
2      UMBF        KEMPER J MARINER  
3      UMBF        KEMPER J MARINER  
4      UMBF        KEMPER J MARINER  
...      ...                    ...  
3140   EXTR       CARINALLI CHARLES  
3141   EXTR       CARINALLI CHARLES  
3142   FRPH   DEVILLIERS DAVID H JR  
3143   FRPH   DEVILLIERS DAVID H JR  
3144   STLD           BUSSE KEITH E  

[3145 rows x 8 columns]

In [17]:
final_summary_df_1[final_summary_df_1['total_amount_bought_or_sold'] > 0]

transaction_date  total_amount_bought_or_sold  \
20          06/17/2021                       1184.0   
31          05/11/2021                       2500.0   
32          05/10/2021                        855.0   
33          05/11/2021                      10000.0   
34          05/19/2021                      40000.0   
...                ...                          ...   
21852       06/17/2021                     296763.0   
21853       05/03/2021                    3731025.0   
21854       06/11/2021                    6268975.0   
21856       03/31/2021                      20000.0   
21858       04/23/2021                       4133.0   

       total_dollar_bought_or_sold  amount_owned_following  \
20                    7.696000e+04                 63339.0   
31                    4.082500e+04                  2500.0   
32                    1.431381e+04                 87751.0   
33                    1.642360e+05                 10000.0   
34                    7.036560e+05                 83760.0   
...                            ...                     ...   
21852                 0.000000e+00               6324895.0   
21853                 7.462050e+06              18672897.0   
21854                 1.253795e+07              18672897.0   
21856                 5.669960e+05                102106.0   
21858                 1.296522e+05                505050.0   

                                                     url reporting_date  \
20     https://sec.gov/Archives/edgar/data/1014282/00...     2021-06-21   
31     https://sec.gov/Archives/edgar/data/1014739/00...     2021-05-12   
32     https://sec.gov/Archives/edgar/data/1014739/00...     2021-05-12   
33     https://sec.gov/Archives/edgar/data/1014739/00...     2021-05-12   
34     https://sec.gov/Archives/edgar/data/1014739/00...     2021-05-20   
...                                                  ...            ...   
21852  https://sec.gov/Archives/edgar/data/1724343/00...     2021-06-21   
21853  https://sec.gov/Archives/edgar/data/1724344/00...     2021-05-20   
21854  https://sec.gov/Archives/edgar/data/1724344/00...     2021-06-11   
21856  https://sec.gov/Archives/edgar/data/1724521/00...     2021-04-02   
21858  https://sec.gov/Archives/edgar/data/1724521/00...     2021-04-26   

       ticker                                             name  purchase_price  
20       NHI                                     WEBB ROBERT T         65.0000  
31      OPCH                                SHAPIRO MICHAEL H.         16.3300  
32      OPCH                                     Pate R Carter         16.7413  
33      OPCH                           RADEMACHER JOHN CHARLES         16.4236  
34      OPCH                         KRAEMER HARRY M JANSEN JR         17.5914  
...       ...                                              ...             ...  
21852   NONE   Fresno County Employees' Retirement Association          0.0000  
21853   ETTX                                    Innoviva, Inc.          2.0000  
21854   ETTX                                    Innoviva, Inc.          2.0000  
21856   RCUS                                 FALBERG KATHRYN E         28.3498  
21858   RCUS                                   KANEKO YASUNORI         31.3700  

[6450 rows x 9 columns]

In [16]:
pd.set_option('max_colwidth', 100) 
pd.set_option('max_rows', 500)

In [17]:
final_summary_df_1

transaction_date  total_amount_bought_or_sold  \
0          05/13/2021                        -4925   
1          05/13/2021                        -5859   
2          05/14/2021                        -5045   
3          05/14/2021                        -4360   
4          05/14/2021                         -304   
...               ...                          ...   
3140       04/30/2021                       -60000   
3141       05/18/2021                       -66745   
3142       05/13/2021                        -1500   
3143       05/18/2021                        -8500   
3144       05/07/2021                      -250000   

      total_dollar_bought_or_sold  amount_owned_following  \
0                         -484051               2023836.0   
1                         -574662               2023836.0   
2                         -495968               2023836.0   
3                         -429782               2023836.0   
4                          -30038               2023836.0   
...                           ...                     ...   
3140                      -692172                410071.0   
3141                      -750753                365071.0   
3142                       -87114                 33358.0   
3143                      -479360                 23358.0   
3144                    -15925606                955041.0   

                                                                              url  \
0      https://sec.gov/Archives/edgar/data/101382/0001127602-21-017085-index.html   
1      https://sec.gov/Archives/edgar/data/101382/0001127602-21-017088-index.html   
2      https://sec.gov/Archives/edgar/data/101382/0001127602-21-017092-index.html   
3      https://sec.gov/Archives/edgar/data/101382/0001127602-21-017096-index.html   
4      https://sec.gov/Archives/edgar/data/101382/0001127602-21-017100-index.html   
...                                                                           ...   
3140  https://sec.gov/Archives/edgar/data/1195839/0001437749-21-010765-index.html   
3141  https://sec.gov/Archives/edgar/data/1195839/0001437749-21-012962-index.html   
3142  https://sec.gov/Archives/edgar/data/1195856/0001474506-21-000110-index.html   
3143  https://sec.gov/Archives/edgar/data/1195856/0001474506-21-000111-index.html   
3144  https://sec.gov/Archives/edgar/data/1195948/0001022671-21-000063-index.html   

     reporting_date  ticker                   name  
0        2021-05-17   UMBF        KEMPER J MARINER  
1        2021-05-17   UMBF        KEMPER J MARINER  
2        2021-05-17   UMBF        KEMPER J MARINER  
3        2021-05-17   UMBF        KEMPER J MARINER  
4        2021-05-17   UMBF        KEMPER J MARINER  
...             ...     ...                    ...  
3140     2021-05-04   EXTR       CARINALLI CHARLES  
3141     2021-05-20   EXTR       CARINALLI CHARLES  
3142     2021-05-18   FRPH   DEVILLIERS DAVID H JR  
3143     2021-05-20   FRPH   DEVILLIERS DAVID H JR  
3144     2021-05-11   STLD           BUSSE KEITH E  

[3145 rows x 8 columns]

In [86]:
final_summary_df_1.to_csv(f'/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Grouped_Insiders/All_Trades/{year}_{quarter}.csv', index=False)

In [87]:
net_buys_1 = final_summary_df_1[final_summary_df_1['total_amount_bought_or_sold'] > 0]

In [88]:
net_buys_1.to_csv(f'/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Grouped_Insiders/Buys_Only/{year}_{quarter}.csv', index=False)

In [89]:
 import smtplib
from email.message import EmailMessage
def send_email(subject, content):
    msg = EmailMessage()
    msg.set_content(content)
    msg['Subject'] = subject
    msg['From'] = 'halpitsstockbot@gmail.com'
    msg['To'] = 'riley.bitterli@gmail.com'

    # Establish a connection to Gmail
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login('[insert your email], '[insert your password]')
    server.send_message(msg)
    server.quit()

send_email('step 2',f' done with {year}')